In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/galaxy-zoo-the-galaxy-challenge/all_ones_benchmark.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/images_test_rev1.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/central_pixel_benchmark.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/images_training_rev1.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/all_zeros_benchmark.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/training_solutions_rev1.zip


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
import cv2
import tensorflow as tf

In [5]:
data = pd.read_csv("../input/galaxy-zoo-the-galaxy-challenge/training_solutions_rev1.zip",
                   compression="zip")

In [6]:
data['GalaxyID'] = data['GalaxyID'].astype(str)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61578 entries, 0 to 61577
Data columns (total 38 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   GalaxyID   61578 non-null  object 
 1   Class1.1   61578 non-null  float64
 2   Class1.2   61578 non-null  float64
 3   Class1.3   61578 non-null  float64
 4   Class2.1   61578 non-null  float64
 5   Class2.2   61578 non-null  float64
 6   Class3.1   61578 non-null  float64
 7   Class3.2   61578 non-null  float64
 8   Class4.1   61578 non-null  float64
 9   Class4.2   61578 non-null  float64
 10  Class5.1   61578 non-null  float64
 11  Class5.2   61578 non-null  float64
 12  Class5.3   61578 non-null  float64
 13  Class5.4   61578 non-null  float64
 14  Class6.1   61578 non-null  float64
 15  Class6.2   61578 non-null  float64
 16  Class7.1   61578 non-null  float64
 17  Class7.2   61578 non-null  float64
 18  Class7.3   61578 non-null  float64
 19  Class8.1   61578 non-null  float64
 20  Class8

In [13]:
data.head()

,GalaxyID,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,...,Class9.3,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6
0,100008,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,...,0.000000,0.279952,0.138445,0.000000,0.000000,0.092886,0.0,0.0,0.0,0.325512
1,100023,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,...,0.018764,0.000000,0.131378,0.459950,0.000000,0.591328,0.0,0.0,0.0,0.000000
2,100053,0.765717,0.177352,0.056931,0.000000,0.177352,0.000000,0.177352,0.000000,0.177352,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
3,100078,0.693377,0.238564,0.068059,0.000000,0.238564,0.109493,0.129071,0.189098,0.049466,...,0.000000,0.094549,0.000000,0.094549,0.189098,0.000000,0.0,0.0,0.0,0.000000
4,100090,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000


In [8]:
import zipfile
with zipfile.ZipFile("../input/galaxy-zoo-the-galaxy-challenge/images_training_rev1.zip", 'r') as zip_ref:
    zip_ref.extractall()

In [9]:
image_dir="/kaggle/working/images_training_rev1"

In [10]:
train_df, val_df = train_test_split(data, test_size=0.2, random_state=42)

In [11]:
class CustomDataGenerator(Sequence):
    def __init__(self, dataframe, image_dir, batch_size=32, target_cols=None, image_size=(224, 224), shuffle=True, is_train=True):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.batch_size = batch_size
        self.target_cols = target_cols
        self.image_size = image_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.dataframe))
        self.is_train = is_train
        
        if self.is_train:
            self.image_data_gen = ImageDataGenerator(
                rescale=1./255,
                rotation_range=20,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                fill_mode='nearest'
            )
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.dataframe) / self.batch_size))

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_df = self.dataframe.iloc[batch_indexes]
        
        X = np.array([self.load_image(row['GalaxyID']) for _, row in batch_df.iterrows()])
        y = batch_df[self.target_cols].values
        
        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def load_image(self, image_name):
        image_path = os.path.join(self.image_dir, f"{image_name}.jpg")  
        image = cv2.imread(image_path)
        image = cv2.resize(image, self.image_size)
        
        if self.is_train:
            image = self.image_data_gen.random_transform(image)
        else:
            # Just rescale for validation images
            image = image / 255.0
                
        return image



In [12]:
batch_size = 256
image_size = (224, 224)  
target_cols = train_df.columns[1:]

In [14]:
train_generator = CustomDataGenerator(train_df, image_dir, batch_size, target_cols, image_size)
val_generator = CustomDataGenerator(val_df, image_dir, batch_size, target_cols, image_size, shuffle=False,is_train=False)

In [15]:
from keras.applications import MobileNetV2
from keras.models import Model,Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow.keras import applications,layers,models

In [16]:
def predictions(model, test_image_paths, batch_size=32, image_size=(224, 224), progress_interval=50):
    predictions = []
    image_ids = []

    for start in range(0, len(test_image_paths), batch_size):
        end = min(start + batch_size, len(test_image_paths))
        batch_paths = test_image_paths[start:end]
        all_images = []

        for image_path in batch_paths:
            image_id = os.path.splitext(os.path.basename(image_path))[0]
            image = cv2.imread(image_path)

            image = cv2.resize(image, image_size)
            image = image / 255.0  
            all_images.append(image)
            image_ids.append(image_id)

        all_images_array = np.array(all_images)
        batch_predictions = model.predict(all_images_array)
        predictions.extend(batch_predictions.tolist())

    return predictions, image_ids



In [17]:
with zipfile.ZipFile("/kaggle/input/galaxy-zoo-the-galaxy-challenge/images_test_rev1.zip", 'r') as zip_ref:
    zip_ref.extractall()

In [18]:
test_image_dir="/kaggle/working/images_test_rev1"

In [19]:
import glob

In [20]:
image_size = (224, 224) 
test_image_paths = glob.glob(os.path.join(test_image_dir, '*.jpg'))

In [21]:
len(test_image_paths)

79975

In [22]:
def save_predictions_to_csv(predictions, image_ids, num_features, filename):
   
    predictions_df = pd.DataFrame(predictions, columns=[f'feature_{i+1}' for i in range(num_features)])
    predictions_df['GalaxyID'] = image_ids 

    predictions_df = predictions_df[['GalaxyID'] + [f'feature_{i+1}' for i in range(num_features)]]

    predictions_df.to_csv(filename, index=False)

In [23]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=2,          
    verbose=1,          
    restore_best_weights=True 
)

Model without FineTuning

In [33]:
def create_model(input_shape=(224,224,3), output_size=37):
    base_model = applications.MobileNetV2(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False 
    
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(), 
        layers.Dense(256, activation='relu'), 
        layers.Dropout(0.5),
        layers.Dense(output_size, activation='linear')  
    ])
    
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    
    return model
    
model = create_model()
model.summary()



Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ ?                      │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,257,984 (8.61 MB)

In [34]:
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=3,  
    verbose=1,
    callbacks=[early_stopping]
)

Epoch 1/3
192/192 ━━━━━━━━━━━━━━━━━━━━ 571s 3s/step - loss: 0.1978 - mae: 0.2332 - val_loss: 0.0416 - val_mae: 0.1217
Epoch 2/3
192/192 ━━━━━━━━━━━━━━━━━━━━ 564s 3s/step - loss: 0.0334 - mae: 0.1137 - val_loss: 0.0357 - val_mae: 0.1166
Epoch 2: early stopping
Restoring model weights from the end of the best epoch: 1.


In [39]:
predictions, image_ids = predictions(model, test_image_paths)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━

In [43]:
save_predictions_to_csv(predictions, image_ids, len(predictions_finetuned[0]), 'predictions_no_finetuning.csv')

Using Model with FineTuning

In [28]:
def create_model_finetuning(input_shape=(224, 224, 3), output_size=37):
    base_model = applications.MobileNetV2(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    
    base_model.trainable = True
    
    for layer in base_model.layers[-10:]:
        layer.trainable = True

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),  
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(output_size, activation='linear') 
    ])
    
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    print(model.summary())
    return model



In [29]:

fine_tuned_model = create_model_finetuning()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ ?                      │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 2,223,872 (8.48 MB)

 Non-trainable params: 34,112 (133.25 KB)

None


In [30]:
fine_tuned_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=3, 
    verbose=1,
    callbacks=[early_stopping]
)

Epoch 1/3


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1730775000.177345      78 service.cc:145] XLA service 0x7b2ffc020600 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730775000.177402      78 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1730775000.177407      78 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
2024-11-05 02:50:11.934475: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng3{k11=2} for conv (f32[256,32,112,112]{3,2,1,0}, u8[0]{0}) custom-call(f32[256,32,112,112]{3,2,1,0}, f32[32,1,3,3]{3,2

192/192 ━━━━━━━━━━━━━━━━━━━━ 675s 3s/step - loss: 0.0770 - mae: 0.1627 - val_loss: 3.3581 - val_mae: 1.4889
Epoch 2/3
192/192 ━━━━━━━━━━━━━━━━━━━━ 571s 3s/step - loss: 0.0189 - mae: 0.0888 - val_loss: 20.6582 - val_mae: 2.8543
Epoch 3/3
192/192 ━━━━━━━━━━━━━━━━━━━━ 571s 3s/step - loss: 0.0167 - mae: 0.0818 - val_loss: 0.0326 - val_mae: 0.1050
Restoring model weights from the end of the best epoch: 3.


In [35]:
predictions_finetuned, image_ids_finetuned = predictions(fine_tuned_model, test_image_paths)

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━

In [42]:
save_predictions_to_csv(predictions_finetuned, image_ids_finetuned, len(predictions_finetuned[0]), 'predictions_finetuning.csv')

FineTuning With LoRA

In [31]:
class LoRALayer(layers.Layer):
    def __init__(self, original_layer, rank=4):
        super(LoRALayer, self).__init__()
        self.original_layer = original_layer
        self.rank = rank
        self.W_r = self.add_weight(shape=(self.original_layer.kernel.shape[0], self.rank), initializer="random_normal", trainable=True)
        self.W_l = self.add_weight(shape=(self.rank, self.original_layer.kernel.shape[1]), initializer="random_normal", trainable=True)

    def call(self, inputs):
        return self.original_layer(inputs) + tf.matmul(tf.matmul(inputs, self.W_r), self.W_l)

def model_with_lora(input_shape=(224, 224, 3), output_size=37, rank=4):
    base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights="imagenet")
    base_model.trainable = False 

    for layer in base_model.layers[-10:]: 
        if isinstance(layer, layers.Conv2D):
            lora_layer = LoRALayer(layer, rank=rank)
            setattr(base_model, layer.name, lora_layer)
            
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(output_size, activation="linear")(x)

    model = Model(inputs=base_model.input, outputs=outputs)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss="mean_squared_error", metrics=["mae"])
    print(model.summary())
    return model

model_with_lora = model_with_lora()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_4[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,595,429 (9.90 MB)

 Trainable params: 337,445 (1.29 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

None


In [32]:
model_with_lora.fit(train_generator, validation_data=val_generator, epochs=3,callbacks=[early_stopping])


Epoch 1/3
192/192 ━━━━━━━━━━━━━━━━━━━━ 573s 3s/step - loss: 0.4677 - mae: 0.4628 - val_loss: 0.1400 - val_mae: 0.2996
Epoch 2/3
192/192 ━━━━━━━━━━━━━━━━━━━━ 564s 3s/step - loss: 0.0576 - mae: 0.1674 - val_loss: 0.1036 - val_mae: 0.2548
Epoch 2: early stopping
Restoring model weights from the end of the best epoch: 1.


In [37]:
predictions_lora, image_ids_lora = predictions(model_with_lora, test_image_paths)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━

In [41]:
save_predictions_to_csv(predictions_lora, image_ids_lora, len(predictions_lora[0]), 'predictions_lora.csv')

In [57]:
file_path = '/kaggle/working/predictions_no_finetuning.csv'
data = pd.read_csv(file_path)
display(data)

,GalaxyID,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37
0,602667,0.265415,0.408663,0.023044,0.048373,0.302545,0.145165,0.270743,0.208920,0.111697,...,0.006904,0.099911,0.070420,0.026323,0.034896,0.110749,-0.008151,0.007219,-0.056443,0.031365
1,741265,0.217159,0.241173,0.026014,0.092056,0.195464,0.071830,0.139856,0.135934,0.136057,...,0.048204,0.049276,0.058762,0.013670,-0.022516,0.076237,-0.020471,-0.008386,0.017462,0.048012
2,385940,0.243058,0.397838,0.034490,0.025111,0.261052,0.167283,0.274334,0.230644,0.077248,...,-0.016119,0.089809,0.073652,-0.000351,0.048158,0.108222,-0.010977,0.018313,-0.077131,0.022160
3,198826,0.130795,0.153875,0.024400,0.080282,0.120000,0.056350,0.126694,0.097330,0.090718,...,0.057715,0.046901,0.043267,0.012202,-0.019191,0.062401,-0.009591,0.012387,0.035276,0.017208
4,432938,0.264979,0.304576,0.024149,0.129096,0.151665,0.109454,0.133583,0.265465,0.232301,...,-0.054037,0.000348,0.132021,0.019186,-0.009549,0.145622,-0.036765,-0.137438,-0.074069,0.112801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79970,501232,0.146525,0.167499,0.020654,0.074917,0.145306,0.050789,0.134116,0.105913,0.089551,...,0.060601,0.059290,0.048121,0.012769,-0.022617,0.059467,-0.003062,0.016365,0.034141,0.018263
79971,303985,0.168452,0.197068,0.019997,0.080464,0.170496,0.056364,0.146330,0.111194,0.097088,...,0.064414,0.060955,0.050025,0.014802,-0.024973,0.059570,-0.003617,0.016883,0.037637,0.020175
79972,410535,0.157456,0.197906,0.015987,0.091521,0.166837,0.059186,0.169857,0.135304,0.090435,...,0.062796,0.057427,0.064776,0.031880,-0.047821,0.060499,0.014233,0.012242,0.054176,0.011850
79973,768825,0.252771,0.325911,0.012577,0.115640,0.276233,0.084703,0.223444,0.159204,0.123610,...,0.077321,0.063316,0.073164,0.039665,-0.060456,0.060607,0.014114,0.015608,0.069249,0.019559


In [58]:
file_path = '/kaggle/working/predictions_finetuning.csv'
data = pd.read_csv(file_path)
display(data)

,GalaxyID,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37
0,602667,0.473988,0.257994,0.085594,0.054725,0.286003,0.036732,0.212533,0.081619,0.187797,...,0.015326,0.035326,0.030547,0.029056,0.005581,0.023137,0.000829,-0.004199,0.000315,0.042080
1,741265,0.474914,0.258277,0.085842,0.054652,0.286375,0.036710,0.212808,0.081616,0.187959,...,0.015261,0.035296,0.030524,0.029015,0.005566,0.023047,0.000811,-0.004218,0.000333,0.042079
2,385940,0.476294,0.258698,0.086211,0.054543,0.286928,0.036677,0.213218,0.081613,0.188201,...,0.015164,0.035252,0.030489,0.028954,0.005543,0.022915,0.000785,-0.004245,0.000361,0.042077
3,198826,0.476318,0.258706,0.086217,0.054541,0.286938,0.036676,0.213225,0.081613,0.188206,...,0.015163,0.035251,0.030488,0.028953,0.005542,0.022912,0.000784,-0.004246,0.000361,0.042077
4,432938,0.474390,0.258117,0.085702,0.054693,0.286164,0.036723,0.212652,0.081618,0.187867,...,0.015298,0.035313,0.030537,0.029038,0.005574,0.023098,0.000821,-0.004207,0.000323,0.042079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79970,501232,0.475323,0.258402,0.085951,0.054619,0.286539,0.036700,0.212929,0.081615,0.188031,...,0.015233,0.035283,0.030513,0.028997,0.005559,0.023008,0.000803,-0.004226,0.000341,0.042078
79971,303985,0.476186,0.258666,0.086182,0.054552,0.286885,0.036679,0.213186,0.081613,0.188182,...,0.015172,0.035255,0.030492,0.028959,0.005544,0.022925,0.000787,-0.004243,0.000359,0.042077
79972,410535,0.476110,0.258642,0.086162,0.054558,0.286854,0.036681,0.213163,0.081613,0.188169,...,0.015177,0.035258,0.030494,0.028962,0.005546,0.022932,0.000788,-0.004241,0.000357,0.042077
79973,768825,0.475300,0.258395,0.085945,0.054621,0.286529,0.036701,0.212923,0.081615,0.188027,...,0.015234,0.035284,0.030514,0.028998,0.005559,0.023010,0.000804,-0.004225,0.000341,0.042078


In [59]:
file_path = '/kaggle/working/predictions_lora.csv'
data = pd.read_csv(file_path)
display(data)

,GalaxyID,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37
0,602667,0.265753,0.747240,-0.039526,-0.175569,0.010436,-0.478961,0.553433,0.507698,0.287868,...,-0.151911,-0.160149,0.228899,-0.187705,0.359097,0.755381,-0.268665,0.394933,-0.359329,0.428149
1,741265,0.049680,0.591976,-0.382434,-0.237265,0.028480,-0.655715,0.477289,0.459242,0.330237,...,0.351419,-0.368854,0.221185,-0.301479,0.249417,0.544604,-0.080615,0.449455,-0.198188,0.231170
2,385940,0.189692,0.466612,0.098635,-0.117405,0.104657,-0.919005,-0.146673,0.418902,0.099512,...,-0.080889,-0.420470,0.131233,-0.327425,0.439873,0.502482,-0.312684,0.066455,-0.131447,0.574113
3,198826,0.341260,0.255322,0.049767,-0.081388,0.130840,-0.433626,0.106745,0.856959,-0.131855,...,0.167064,-0.070615,0.422044,-0.185647,0.152975,0.757533,-0.100651,-0.011657,0.175712,0.365524
4,432938,0.157078,0.214050,-0.070561,0.153415,0.029605,-0.692855,0.049252,0.400390,0.061232,...,0.036260,-0.185407,0.712398,-0.287608,0.339054,0.792292,-0.127132,0.185986,-0.005181,0.740871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79970,501232,0.251283,0.403938,-0.325534,-0.427408,0.144982,-0.622746,0.493786,0.736762,0.060089,...,0.022043,-0.345616,0.112107,-0.428651,0.344849,0.956773,-0.268808,0.355591,-0.342885,0.521986
79971,303985,-0.133819,0.525835,-0.356925,-0.138345,0.239276,-0.578703,0.617243,0.319703,0.309892,...,0.307664,-0.126887,0.359141,-0.245486,0.235441,0.396374,-0.054452,0.367118,-0.306850,0.437130
79972,410535,-0.144244,0.372297,-0.231467,0.086271,0.034020,-0.548045,0.606628,0.252644,0.512594,...,0.182215,-0.011108,0.391063,-0.175108,0.138445,0.313684,0.141221,0.413350,-0.546968,0.431526
79973,768825,-0.232277,0.718629,-0.090865,-0.251560,0.286494,-0.719646,0.750586,0.161429,0.443967,...,0.116344,-0.053598,0.390093,-0.404390,0.423864,0.367664,0.210568,0.362587,-0.459124,0.181444
